In [1]:
import requests
import shutil
import os
import io
import json
import time
import datetime
import dateutil
import random
import re
import ast
import pickle
import numpy as np
import pandas as pd
from IPython.display import clear_output
from pprint import pprint
import matplotlib.pyplot as plt

parent_folder='steam_data'

csv_folder=os.path.join(parent_folder,"bulk","csv_no_text","early_access_standardized")
csv_english_folder=os.path.join(parent_folder,"bulk","csv_english","early_access_standardized")
testing_folder=os.path.join(parent_folder,"bulk","testing")

In [2]:
with open(os.path.join(testing_folder,'still_in_early_access_FINAL'),'r') as file:
    current_ea_games=sorted(list(set([int(x) for x in file.read().split('\n')[:-1]])))
with open(os.path.join(testing_folder,'out_of_early_access_FINAL'),'r') as file:
    former_ea_games=sorted(list(set([int(x) for x in file.read().split('\n')[:-1]])))
full_games_list=sorted(list(set(current_ea_games+former_ea_games)))
with open(os.path.join(parent_folder,'official_appid_json_list'),'r') as file:
    temp_json=json.load(file)['applist']['apps']
official_app_list={}
for x in temp_json:
    official_app_list[x['appid']]=x['name']
print(len(full_games_list))
game_df=pd.read_csv(os.path.join(testing_folder,'20190124_games.csv'))
all_categories=set()
all_genres=set()
all_developers={}
all_publishers={}
for x in ['genres','developers','publishers','categories']:
    game_df[x]=game_df[x].apply(ast.literal_eval)
for i,row in game_df.iterrows():
    all_categories=all_categories | set(row['categories'])
    all_genres=all_genres | set(row['genres'])
    #print(all_genres)
    for developer in row['developers']:
        if developer not in all_developers:
            all_developers[developer]=1
        else:
            all_developers[developer]+=1
    for publisher in row['publishers']:
        if publisher not in all_publishers:
            all_publishers[publisher]=1
        else:
            all_publishers[publisher]+=1
#print(all_genres)
all_genres.remove('Early Access')
all_categories=sorted(list(all_categories))
all_genres=sorted(list(all_genres))

4935


In [3]:
num_reviews_df=pd.read_csv(os.path.join(testing_folder,'20190126_num_reviews.csv')).set_index('appid')
early_access_start_time_df=pd.read_csv(os.path.join(testing_folder,'20190126_early_access_start_time.csv')).set_index('appid')
normal_review_start_time_df=pd.read_csv(os.path.join(testing_folder,'20190126_normal_review_start_time.csv')).set_index('appid')
num_english_reviews={}

In [48]:
csv_folder=os.path.join(parent_folder,"bulk","csv_english","early_access_standardized")
out_folder=os.path.join(parent_folder,"bulk","csv_english","time_series_by_month_revised")
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

for i,appid in enumerate(current_ea_games,1):
    if appid in num_reviews_df.index:
        temp_time_series=[]
        try:
            df=pd.read_csv(os.path.join(csv_folder,str(appid)+'_reviews_english.csv'))
            english_file_exists=True
        except:
            english_file_exists=False
        if english_file_exists:
            df['num_reviews']=df['num_reviews'].apply(lambda x: max(1,x))
            df['num_games_owned']=df['num_games_owned'].apply(lambda x: max(1,x))
            num_english_reviews[int(appid)]=len(df)
            earliest_time=early_access_start_time_df.loc[int(appid)]['time']
            latest_time=df['timestamp_updated'].max()
            num_months=int(np.ceil(float(latest_time-earliest_time+1)/(30.0*24.0*60.0*60.0)))
            for j in range(num_months):
                df_pos=df[(df['timestamp_updated']>=earliest_time+(j)*30.0*24.0*60.0*60.0) &
                            (df['timestamp_updated']<earliest_time+(j+1)*30.0*24.0*60.0*60.0) &
                              (df['voted_up']==True)]
                df_neg=df[(df['timestamp_updated']>=earliest_time+(j)*30.0*24.0*60.0*60.0) &
                            (df['timestamp_updated']<earliest_time+(j+1)*30.0*24.0*60.0*60.0) &
                              (df['voted_up']==False)]
                num_pos=float(len(df_pos))
                num_neg=float(len(df_neg))
                weight_likes_pos=num_pos+df_pos['votes_up'].sum()
                weight_likes_neg=num_neg+df_neg['votes_up'].sum()
                weight_games_pos=df_pos['num_games_owned'].sum()
                weight_games_neg=df_neg['num_games_owned'].sum()
                weight_reviews_pos=df_pos['num_reviews'].sum()
                weight_reviews_neg=df_neg['num_reviews'].sum()
                if num_neg>0 or num_pos>0:
                    temp_time_series.append({'early_access':True,'month':j,'reviews':int(num_pos+num_neg),
                                             'ratio':num_pos/(num_neg+num_pos),'weight_likes_ratio':weight_likes_pos/(weight_likes_pos+weight_likes_neg),
                                            'weight_games_ratio':weight_games_pos/(weight_games_pos+weight_games_neg),
                                            'weight_reviews_ratio':weight_reviews_pos/(weight_reviews_pos+weight_reviews_neg)})
            pd.DataFrame(temp_time_series).to_csv(os.path.join(out_folder,str(appid)+'_month_time_series.csv'),index=False)
        clear_output(wait=True)
        print(str(i)+"/"+str(len(current_ea_games)))

2939/2939


In [47]:
csv_folder=os.path.join(parent_folder,"bulk","csv_english","early_access_standardized")
out_folder=os.path.join(parent_folder,"bulk","csv_english","time_series_by_month_revised")
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

for i,appid in enumerate(former_ea_games,1):
    if appid in num_reviews_df.index:
        try:
            df=pd.read_csv(os.path.join(csv_folder,str(appid)+'_reviews_english.csv'))
            english_file_exists=True
        except:
            english_file_exists=False
        if english_file_exists:
            df['num_reviews']=df['num_reviews'].apply(lambda x: max(1,x))
            df['num_games_owned']=df['num_games_owned'].apply(lambda x: max(1,x))
            if len(df[df['written_during_early_access']==True])>0 and len(df[df['written_during_early_access']==False])>0:
                temp_time_series=[]
                num_english_reviews[int(appid)]=len(df)
                earliest_time=early_access_start_time_df.loc[int(appid)]['time']
                normal_review_time=normal_review_start_time_df.loc[int(appid)]['time']
                latest_time=df['timestamp_updated'].max()

                temp_time_series=[]
                num_months=int(np.ceil(float(normal_review_time-earliest_time)/(30.0*24.0*60.0*60.0)))
                for j in range(num_months):
                    df_pos=df[(df['timestamp_updated']>=earliest_time+(j)*30.0*24.0*60.0*60.0) &
                                (df['timestamp_updated']<min(earliest_time+(j+1)*30.0*24.0*60.0*60.0,normal_review_time)) &
                                  (df['voted_up']==True)]
                    df_neg=df[(df['timestamp_updated']>=earliest_time+(j)*30.0*24.0*60.0*60.0) &
                                (df['timestamp_updated']<min(earliest_time+(j+1)*30.0*24.0*60.0*60.0,normal_review_time)) &
                                  (df['voted_up']==False)]
                    num_pos=float(len(df_pos))
                    num_neg=float(len(df_neg))
                    weight_likes_pos=num_pos+df_pos['votes_up'].sum()
                    weight_likes_neg=num_neg+df_neg['votes_up'].sum()
                    weight_games_pos=df_pos['num_games_owned'].sum()
                    weight_games_neg=df_neg['num_games_owned'].sum()
                    weight_reviews_pos=df_pos['num_reviews'].sum()
                    weight_reviews_neg=df_neg['num_reviews'].sum()
                    if num_neg>0 or num_pos>0:
                        temp_time_series.append({'early_access':True,'month':j,'reviews':int(num_pos+num_neg),
                                                 'ratio':num_pos/(num_neg+num_pos),'weight_likes_ratio':weight_likes_pos/(weight_likes_pos+weight_likes_neg),
                                                'weight_games_ratio':weight_games_pos/(weight_games_pos+weight_games_neg),
                                                'weight_reviews_ratio':weight_reviews_pos/(weight_reviews_pos+weight_reviews_neg)})
                pd.DataFrame(temp_time_series).to_csv(os.path.join(out_folder,str(appid)+'_month_time_series_early.csv'),index=False)

                temp_time_series=[]
                num_months=int(np.ceil(float(latest_time-normal_review_time+1)/(30.0*24.0*60.0*60.0)))
                for j in range(num_months):
                    df_pos=df[(df['timestamp_updated']>=normal_review_time+(j)*30.0*24.0*60.0*60.0) &
                                (df['timestamp_updated']<normal_review_time+(j+1)*30.0*24.0*60.0*60.0) &
                                  (df['voted_up']==True)]
                    df_neg=df[(df['timestamp_updated']>=normal_review_time+(j)*30.0*24.0*60.0*60.0) &
                                (df['timestamp_updated']<normal_review_time+(j+1)*30.0*24.0*60.0*60.0) &
                                  (df['voted_up']==False)]
                    num_pos=float(len(df_pos))
                    num_neg=float(len(df_neg))
                    weight_likes_pos=num_pos+df_pos['votes_up'].sum()
                    weight_likes_neg=num_neg+df_neg['votes_up'].sum()
                    weight_games_pos=df_pos['num_games_owned'].sum()
                    weight_games_neg=df_neg['num_games_owned'].sum()
                    weight_reviews_pos=df_pos['num_reviews'].sum()
                    weight_reviews_neg=df_neg['num_reviews'].sum()
                    if num_neg>0 or num_pos>0:
                        temp_time_series.append({'early_access':False,'month':j,'reviews':int(num_pos+num_neg),
                                                 'ratio':num_pos/(num_neg+num_pos),'weight_likes_ratio':weight_likes_pos/(weight_likes_pos+weight_likes_neg),
                                                'weight_games_ratio':weight_games_pos/(weight_games_pos+weight_games_neg),
                                                'weight_reviews_ratio':weight_reviews_pos/(weight_reviews_pos+weight_reviews_neg)})
                pd.DataFrame(temp_time_series).to_csv(os.path.join(out_folder,str(appid)+'_month_time_series_release.csv'),index=False)



                clear_output(wait=True)
                print(str(i)+"/"+str(len(former_ea_games)))

1996/1996


In [104]:
csv_folder=os.path.join(parent_folder,"bulk","csv_english","early_access_standardized")
out_folder=os.path.join(parent_folder,"bulk","csv_english","former_early_access_text_cleaned_with_periods_2")
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),.!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].apply(lambda x: re.sub('\n+','. ',x))
    df[text_field] = df[text_field].apply(lambda x: re.sub('\.\.\.+','. ',x))
    df[text_field] = df[text_field].apply(lambda x: re.sub('<br ?/?>+','. ',x))
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.lower()
    return df

for i,appid in enumerate(former_ea_games,1):
    if appid in num_reviews_df.index:
        try:
            df=pd.read_csv(os.path.join(csv_folder,str(appid)+'_reviews_english.csv')).dropna()
            english_file_exists=True
        except:
            english_file_exists=False
        if english_file_exists:
            if len(df[df['written_during_early_access']==True])>0 and len(df[df['written_during_early_access']==False])>0:
                df=standardize_text(df,'review')   
                df.to_csv(os.path.join(out_folder,str(appid)+'_reviews_english.csv'),index=False)
                clear_output(wait=True)
                print(str(i)+"/"+str(len(former_ea_games)))

1996/1996


In [107]:
csv_folder=os.path.join(parent_folder,"bulk","csv_english","former_early_access_text_cleaned_with_periods_2")
out_folder=os.path.join(parent_folder,"bulk","csv_english","former_early_access_text_cleaned_with_periods_2")
#if not os.path.exists(out_folder):
#    os.makedirs(out_folder)

df_list=[]
for i,appid in enumerate(former_ea_games,1):
    if appid in num_reviews_df.index:
        try:
            df_list.append(pd.read_csv(os.path.join(csv_folder,str(appid)+'_reviews_english.csv')))
            english_file_exists=True
        except:
            english_file_exists=False
        clear_output(wait=True)
        print(str(i)+"/"+str(len(former_ea_games)))
df=pd.concat(df_list).dropna()
df.to_csv(os.path.join(out_folder,'all_reviews_english.csv'),index=False)
print("Done!")

1996/1996
Done!


In [39]:
csv_folder=os.path.join(parent_folder,"bulk","csv_english","former_early_access_text_cleaned_with_periods_2")
df=pd.read_csv(os.path.join(csv_folder,'all_reviews_english.csv'))

In [110]:
#import nltk
#for i in range(10):
#    print(i)
#    print(df.iloc[i]['appid'])
#    print(nltk.tokenize.sent_tokenize(df.iloc[i]['review']))
df["sentence_lists"] = df["review"].apply(nltk.tokenize.sent_tokenize)
#print(nltk.tokenize.sent_tokenize(df.iloc[4]['review']))
#print(nltk.tokenize.sent_tokenize("i loved this game until the newer alphas came  now i can't play this game   please bring me back to the july 2 2015 alpha   \n\nnow when i kick an enemy the game crashes   \n\n9 years of development   \nupdate its eepic furry huntin sim".replace('\n','. ')))

In [112]:
df.to_pickle(os.path.join(csv_folder,'all_reviews_sentence_tokenized.pkl'))

In [114]:
df.drop('review',axis=1).to_pickle(os.path.join(csv_folder,'all_reviews_sentences_only_tokenized.pkl'))

In [ ]:
df=pd.read_pickle()

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def cv(data):
    count_vectorizer = CountVectorizer()

    emb = count_vectorizer.fit_transform(data)

    return emb, count_vectorizer

print(time.time())
list_corpus = df['review'].dropna().tolist()
print(time.time())
#list_labels = clean_questions["class_label"].tolist()

#X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, test_size=0.2, 
#                                                                                random_state=40)

X_train_counts, original_count_vectorizer = cv(list_corpus)
print(time.time()) # TAKES ABOUT 1.5 MINUTES
print("DONE")
#X_test_counts = count_vectorizer.transform(X_test)

1548643915.6949265
1548643916.1312172
1548644009.4807522
DONE


In [14]:
from collections import Counter

X_test_counts = original_count_vectorizer.transform(["testing"])


vocab = list(original_count_vectorizer.get_feature_names())
counts = X_test_counts.sum(axis=0).A1
freq_distribution = Counter(dict(zip(vocab, counts)))
for x in freq_distribution.most_common(50):
    print(x)

('be', 1)
('just', 1)
('ot', 1)
('please', 1)
('working', 1)
('00', 0)
('000', 0)
('0000', 0)
('00000', 0)
('000000', 0)
('0000000', 0)
('00000000', 0)
('000000000', 0)
('0000000000', 0)
('00000000000', 0)
('00000000000000', 0)
('000000000000000', 0)
('0000000000000000', 0)
('000000000000000000', 0)
('0000000000000000000', 0)
('00000000000000000000', 0)
('000000000000000000000000', 0)
('0000000000000000000000000', 0)
('00000000000000000000000000000', 0)
('000000000000000000000000000000', 0)
('0000000000000000000000000000000', 0)
('00000000000000000000000000000000', 0)
('0000000000000000000000000000000000', 0)
('00000000000000000000000000000000000', 0)
('000000000000000000000000000000000000', 0)
('00000000000000000000000000000000000000', 0)
('000000000000000000000000000000000000000', 0)
('00000000000000000000000000000000000000000', 0)
('0000000000000000000000000000000000000000000', 0)
('00000000000000000000000000000000000000000000', 0)
('000000000000000000000000000000000000000000000000'

In [45]:

from collections import Counter

vocab = list(count_vectorizer.get_feature_names())

# Column-wise sum of the X matrix.
# It's some crazy numpy syntax that looks horribly unpythonic
# For details, see http://stackoverflow.com/questions/3337301/numpy-matrix-to-array
# and http://stackoverflow.com/questions/13567345/how-to-calculate-the-sum-of-all-columns-of-a-2d-numpy-array-efficiently
counts = X_train_counts.sum(axis=0).A1

freq_distribution = Counter(dict(zip(vocab, counts)))


[('the', 5925490), ('and', 4012497), ('to', 3775248), ('game', 3608955), ('it', 3035036), ('you', 2655546), ('of', 2550252), ('is', 2546335), ('this', 2194596), ('in', 1633458), ('that', 1431660), ('for', 1369493), ('but', 1251729), ('with', 1176772), ('have', 924430), ('on', 912279), ('are', 907641), ('can', 825614), ('not', 804966), ('be', 780810), ('like', 743585), ('if', 736318), ('as', 712006), ('so', 697460), ('your', 656153), ('my', 637023), ('just', 595857), ('or', 587089), ('fun', 577877), ('play', 576923), ('get', 569643), ('good', 559342), ('more', 555710), ('there', 550055), ('all', 544580), ('they', 544040), ('was', 499163), ('at', 497660), ('will', 463113), ('very', 452767), ('10', 445457), ('one', 441331), ('its', 429557), ('some', 422220), ('really', 422154), ('me', 419025), ('time', 416435), ('great', 408579), ('out', 406419), ('has', 403614), ('an', 400412), ('from', 391109), ('no', 376534), ('would', 371067), ('when', 369773), ('up', 367049), ('do', 364986), ('games'

In [49]:
for x in freq_distribution.most_common(2000):
    print(x)

('the', 5925490)
('and', 4012497)
('to', 3775248)
('game', 3608955)
('it', 3035036)
('you', 2655546)
('of', 2550252)
('is', 2546335)
('this', 2194596)
('in', 1633458)
('that', 1431660)
('for', 1369493)
('but', 1251729)
('with', 1176772)
('have', 924430)
('on', 912279)
('are', 907641)
('can', 825614)
('not', 804966)
('be', 780810)
('like', 743585)
('if', 736318)
('as', 712006)
('so', 697460)
('your', 656153)
('my', 637023)
('just', 595857)
('or', 587089)
('fun', 577877)
('play', 576923)
('get', 569643)
('good', 559342)
('more', 555710)
('there', 550055)
('all', 544580)
('they', 544040)
('was', 499163)
('at', 497660)
('will', 463113)
('very', 452767)
('10', 445457)
('one', 441331)
('its', 429557)
('some', 422220)
('really', 422154)
('me', 419025)
('time', 416435)
('great', 408579)
('out', 406419)
('has', 403614)
('an', 400412)
('from', 391109)
('no', 376534)
('would', 371067)
('when', 369773)
('up', 367049)
('do', 364986)
('games', 362103)
('even', 357008)
('what', 335490)
('only', 32235

In [55]:
print('was' in count_vectorizer.get_feature_names())

True


In [37]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
####


##########
all_feature_word_lists=[['devs', 'dev', 'developers', 'development', 'develop', 'developer', 'developed'],
["update", 'updates', 'updated', 'updating'],
['new'],
['bug', 'bugs', 'buggy', 'glitch', 'glitchy', 'glitches'],
['access', 'beta'],
['refund'],
['content'],
['cash', 'money', "dollars"],
['community', 'playerbase', 'players'],
['care'],
['ignore'],
['release', 'released'],
['month', 'months'],
['year', 'years'],
['improve', 'improved', 'improvement', 'improvements'],
['balance', 'balanced', 'unbalanced', 'balancing'],
['unplayable'],
['broken']]

all_feature_words=[]
for word_list in all_feature_word_lists:
    all_feature_words+=word_list
fake_sentence=" ".join(all_feature_words)
spec_count_vectorizer=CountVectorizer()
X=spec_count_vectorizer.fit_transform([fake_sentence])
vocab=list(spec_count_vectorizer.get_feature_names())

all_feature_idx_lists=[[vocab.index(x) for x in y] for y in all_feature_word_lists]

def add_count_features(v,df,fidx):
    for ri in range(len(df)):
        X = spec_count_vectorizer.transform([df.iloc[ri]['review']])
        counts=X.sum(axis=0).A1
        for rj,idx_list in enumerate(all_feature_idx_lists,0):
            for idx in idx_list:
                v[fidx+rj]+=counts[idx]
                v[fidx+rj+len(all_feature_idx_lists)]+=counts[idx]*df.iloc[ri]['votes_up']
        #break
print("WAIT")       

csv_folder=os.path.join(parent_folder,"bulk","csv_english","former_early_access_text_cleaned")
out_folder=os.path.join(parent_folder,"bulk","csv_english","former_early_access_text_cleaned_count_vecs")
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

initial_time=time.time()

for i,appid in enumerate(former_ea_games,1):
    if appid in num_reviews_df.index:
        try:
            df=pd.read_csv(os.path.join(csv_folder,str(appid)+'_reviews_english.csv')).dropna()
            english_file_exists=True
        except:
            english_file_exists=False
        if english_file_exists:
            df=df[df['written_during_early_access']==True]
            clear_output(wait=True)
            print(str(i)+"/"+str(len(former_ea_games))+": "+str(len(df))+" reviews ("+str(int((time.time()-initial_time)/60))+" minutes so far)")
            df_pos=df[df['voted_up']==True]
            df_neg=df[df['voted_up']!=True]
            if len(df)>0:
                feature_v=np.zeros(len(all_feature_idx_lists)*4)
                add_count_features(feature_v,df_pos,0)
                add_count_features(feature_v,df_neg,len(all_feature_idx_lists)*2)
        np.savetxt(os.path.join(out_folder,str(appid)+"_count_vec.csv"), feature_v, delimiter=",")
        
print("Done!")
#df=pd.concat(df_list)
#df.to_csv(os.path.join(out_folder,'all_reviews_english.csv'),index=False)



1996/1996: 2 reviews (165 minutes so far)
Done!


In [64]:
del df

In [85]:
import re

print(re.sub(r'\n+', '. ', 'a\n\n\nb'))

a. b
